In [1]:
import pandas as pd
import numpy as np
import requests
import os 
from dotenv import load_dotenv
import requests
from collections import Counter
from fuzzywuzzy import process, fuzz
from tqdm import tqdm
import pickle
import re
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.max_columns = None
pd.set_option('display.float_format', lambda x: '%.3f' % x)

import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.append("../")

import src.biblioteca as bb

Note to self:
en este archivo van cosas que luego van a ser tres archivos distintos.
- Limpieza
- EDA
- Charts
- 
Recomendable separarlo y trabajarlo de forma independiente

# Input files

Base data file

In [2]:
with open('../data/pickle/dataset_usa_base.pickle', 'rb') as base_data:
    base_data = pickle.load(base_data)
print(base_data.shape)
base_data.head(1)

(455057, 9)


,title,rank,date,artist,url,region,chart,trend,streams
8246,Bad and Boujee (feat. Lil Uzi Vert),1,2017-01-01,Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,United States,top200,SAME_POSITION,1371493.000


LastFM extraction

In [3]:
with open('../data/pickle/spotify_dataset_lastfm.pickle', 'rb') as data_lastfm:
    data_lastfm = pickle.load(data_lastfm)
data_lastfm.drop_duplicates(inplace=True)
print(data_lastfm.drop_duplicates().shape)
data_lastfm.head(1)

(14659, 17)


,url,artist,track,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4
8246,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,migos,bad and boujee (feat. lil uzi vert),"30 Jan 2018, 04:41","26 Jun 2013, 14:20","Migos is a rap trio-group from Lawrenceville, ...","Migos is a rap trio-group from Lawrenceville, ...",Hip-Hop,trap,rap,hip hop,atlanta,trap,Hip-Hop,rap,2010s,meme rap


Spotify extraction

In [4]:
with open('../data/pickle/spotify_dataset_audioFeatures.pickle', 'rb') as data_spotify:
    data_spotify = pickle.load(data_spotify)
data_spotify.drop_duplicates(inplace=True)
print(data_spotify.drop_duplicates().shape)
data_spotify.head(1)

(13989, 21)


,url,artist,track,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature
0,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,migos,bad and boujee (feat. lil uzi vert),spotify:track:0M9ydKzuF3oZTfYYPfaGX1,0.926,0.666,11.000,-5.314,1.000,0.244,0.061,0.000,0.123,0.168,127.079,audio_features,0M9ydKzuF3oZTfYYPfaGX1,https://api.spotify.com/v1/tracks/0M9ydKzuF3oZ...,https://api.spotify.com/v1/audio-analysis/0M9y...,343150.000,4.000


# Merging all data

In [5]:
df = base_data.merge(data_lastfm, on = 'url', indicator = True, how = 'left')
df.value_counts('_merge')

_merge
both          456306
left_only          0
right_only         0
dtype: int64

In [6]:
df.drop(['_merge'], axis = 1, inplace = True)
df = df.merge(data_spotify, on = 'url', indicator = True, how = 'left')
df.value_counts('_merge')

_merge
both          442906
left_only      13924
right_only         0
dtype: int64

In [7]:
df.drop(['artist_y', 'track_x', 'artist', 'track_y', '_merge'], axis = 1, inplace= True)
df.rename({'artist_x':'artist'}, inplace=True, axis=1)
print(df.shape)
df.head(1)

(456830, 41)


,title,rank,date,artist,url,region,chart,trend,streams,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature
0,Bad and Boujee (feat. Lil Uzi Vert),1,2017-01-01,Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,United States,top200,SAME_POSITION,1371493.000,"30 Jan 2018, 04:41","26 Jun 2013, 14:20","Migos is a rap trio-group from Lawrenceville, ...","Migos is a rap trio-group from Lawrenceville, ...",Hip-Hop,trap,rap,hip hop,atlanta,trap,Hip-Hop,rap,2010s,meme rap,spotify:track:0M9ydKzuF3oZTfYYPfaGX1,0.926,0.666,11.000,-5.314,1.000,0.244,0.061,0.000,0.123,0.168,127.079,audio_features,0M9ydKzuF3oZTfYYPfaGX1,https://api.spotify.com/v1/tracks/0M9ydKzuF3oZ...,https://api.spotify.com/v1/audio-analysis/0M9y...,343150.000,4.000


# Cleaning data

In [8]:
df.head(1)

,title,rank,date,artist,url,region,chart,trend,streams,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature
0,Bad and Boujee (feat. Lil Uzi Vert),1,2017-01-01,Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,United States,top200,SAME_POSITION,1371493.000,"30 Jan 2018, 04:41","26 Jun 2013, 14:20","Migos is a rap trio-group from Lawrenceville, ...","Migos is a rap trio-group from Lawrenceville, ...",Hip-Hop,trap,rap,hip hop,atlanta,trap,Hip-Hop,rap,2010s,meme rap,spotify:track:0M9ydKzuF3oZTfYYPfaGX1,0.926,0.666,11.000,-5.314,1.000,0.244,0.061,0.000,0.123,0.168,127.079,audio_features,0M9ydKzuF3oZTfYYPfaGX1,https://api.spotify.com/v1/tracks/0M9ydKzuF3oZ...,https://api.spotify.com/v1/audio-analysis/0M9y...,343150.000,4.000


Highest rank in list + Weeks in charts

In [9]:
lista_url = df['url'].unique().tolist()
len(lista_url)

14650

In [10]:
ranks = {
    'url':[],
    'highest_rank':[],
    'weeks_in_charts':[]
}

In [11]:
for url in tqdm(lista_url):
    ranks['url'].append(url)
    ranks['highest_rank'].append(df[df['url'] == url].sort_values(by = 'rank').iloc[0,1])
    ranks['weeks_in_charts'].append(df[df['url'] == url].shape[0])

100%|██████████| 14650/14650 [14:04<00:00, 17.34it/s]


In [12]:
print(len(ranks['url']), len(ranks['highest_rank']), len(ranks['weeks_in_charts']))

14650 14650 14650


In [13]:
df = df.merge(pd.DataFrame(ranks), on = 'url', indicator= True)
df.value_counts('_merge')

_merge
both          456830
left_only          0
right_only         0
dtype: int64

In [14]:
df.drop(['_merge'], axis = 1, inplace=True)

Main music genre

Primo los géneros de las canciones -- si no hubiera datos, cojo los valores del artista

In [15]:
df.head(1)

,title,rank,date,artist,url,region,chart,trend,streams,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,highest_rank,weeks_in_charts
0,Bad and Boujee (feat. Lil Uzi Vert),1,2017-01-01,Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,United States,top200,SAME_POSITION,1371493.000,"30 Jan 2018, 04:41","26 Jun 2013, 14:20","Migos is a rap trio-group from Lawrenceville, ...","Migos is a rap trio-group from Lawrenceville, ...",Hip-Hop,trap,rap,hip hop,atlanta,trap,Hip-Hop,rap,2010s,meme rap,spotify:track:0M9ydKzuF3oZTfYYPfaGX1,0.926,0.666,11.000,-5.314,1.000,0.244,0.061,0.000,0.123,0.168,127.079,audio_features,0M9ydKzuF3oZTfYYPfaGX1,https://api.spotify.com/v1/tracks/0M9ydKzuF3oZ...,https://api.spotify.com/v1/audio-analysis/0M9y...,343150.000,4.000,1,429


In [16]:
df['music_genre'] = np.where(df['aritist_genre_0'].isnull() == True, df['track_genre_0'].str.lower(), df['aritist_genre_0'].str.lower())
df['music_genre'].value_counts()

hip-hop         131641
pop              65166
rap              30438
country          25113
rnb              22598
                 ...  
wave                 1
calypso              1
southern rap         1
salsa                1
c                    1
Name: music_genre, Length: 363, dtype: int64

In [17]:
cuenta_generos = Counter(genres for genres in df['music_genre'])
print(f"Hay {len(cuenta_generos)} generos diferentes.")

Hay 2976 generos diferentes.


In [18]:
dict_genres = dict(cuenta_generos.most_common(24))

In [19]:
fuzz.ratio('hip-hop', "hip hop")

86

In [20]:
def music_genres(column, genres):
    max = 0
    for key in genres.keys():
        try:
            similar = fuzz.ratio(column, key)
            if similar > max:
                max = similar
                genre = key     
        except:
            return "other"
    if max > 60:
        return genre
    else:
        return "other"

In [21]:
df["clean_music_genre"] = df.apply(lambda x: music_genres(x["music_genre"], dict_genres), axis = 1)
df["clean_music_genre"].value_counts()

hip-hop             132882
other                91631
pop                  65166
rap                  30446
country              25295
rnb                  22649
trap                 19979
electronic           11784
indie                11121
rock                  6456
soul                  5459
acoustic              3902
indie pop             3526
classic rock          3410
jazz                  3110
k-pop                 2783
house                 2610
lo-fi                 2416
indie rock            2098
latin                 2065
female vocalists      1823
black metal           1772
cloud rap             1600
reggaeton             1443
seen live             1404
Name: clean_music_genre, dtype: int64

Gender from artists

ojo, hay un poco de error -- volver a pasarlo, pero solo con el nombre de los artistas (que no sean grupos)

In [22]:
df.sample(5)

,title,rank,date,artist,url,region,chart,trend,streams,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,highest_rank,weeks_in_charts,music_genre,clean_music_genre
146441,F.N,38,2019-11-13,Lil Tjay,https://open.spotify.com/track/0RLLvYBopDwooCy...,United States,top200,MOVE_DOWN,458205.000,"08 May 2021, 22:38","28 Sep 2019, 22:26","Tione Kenyon Jayden Merritt (born April 30, 20...","Tione Kenyon Jayden Merritt (born April 30, 20...",rap,Hip-Hop,new york,hip hop,american,chigga,NaN,NaN,NaN,NaN,spotify:track:0RLLvYBopDwooCyvnTwgYu,0.567,0.515,6.000,-10.265,0.000,0.247,0.354,0.000,0.078,0.321,120.100,audio_features,0RLLvYBopDwooCyvnTwgYu,https://api.spotify.com/v1/tracks/0RLLvYBopDwo...,https://api.spotify.com/v1/audio-analysis/0RLL...,224000.000,1.000,21,397,rap,rap
342199,Viagra,3,2019-05-20,"ITSOKTOCRY, 909memphis",https://open.spotify.com/track/48ekYKpqr6tq2Bt...,United States,viral50,MOVE_UP,NaN,NaN,"01 Jan 1970, 00:00","<a href=""https://www.last.fm/music/ITSOKTOCRY...",,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,spotify:track:48ekYKpqr6tq2Btal2vFhk,0.730,0.463,7.000,-9.461,1.000,0.124,0.001,0.000,0.108,0.143,156.895,audio_features,48ekYKpqr6tq2Btal2vFhk,https://api.spotify.com/v1/tracks/48ekYKpqr6tq...,https://api.spotify.com/v1/audio-analysis/48ek...,159000.000,4.000,1,29,NaN,other
148071,JUMP (feat. YoungBoy Never Broke Again),174,2020-05-09,DaBaby,https://open.spotify.com/track/0oT9ElXYSxvnOOa...,United States,top200,MOVE_UP,228192.000,NaN,"31 Mar 2019, 05:03","DaBaby was born Jonathan Kirk in Cleveland, Oh...","DaBaby was born Jonathan Kirk in Cleveland, Oh...",Hip-Hop,rap,trap,hip hop,american,NaN,NaN,NaN,NaN,NaN,spotify:track:0oT9ElXYSxvnOOagP9efDq,0.896,0.720,0.000,-6.262,0.000,0.355,0.169,0.000,0.252,0.966,140.100,audio_features,0oT9ElXYSxvnOOagP9efDq,https://api.spotify.com/v1/tracks/0oT9ElXYSxvn...,https://api.spotify.com/v1/audio-analysis/0oT9...,212093.000,4.000,11,58,hip-hop,hip-hop
307435,Rich Love (with Seeb),112,2017-09-18,OneRepublic,https://open.spotify.com/track/7aF09WaavZAmAWu...,United States,top200,MOVE_UP,247605.000,NaN,"14 May 2006, 03:52","OneRepublic is an American, self-proclaimed ""g...","OneRepublic is an American, self-proclaimed ""g...",rock,pop rock,alternative,alternative rock,indie,NaN,NaN,NaN,NaN,NaN,spotify:track:7dOeiXeTSfA1ixaYmQcWu7,0.401,0.872,1.000,-3.641,0.000,0.314,0.167,0.000,0.362,0.472,104.592,audio_features,7dOeiXeTSfA1ixaYmQcWu7,https://api.spotify.com/v1/tracks/7dOeiXeTSfA1...,https://api.spotify.com/v1/audio-analysis/7dOe...,201257.000,4.000,23,88,rock,rock
399849,Dangerous,63,2021-02-05,Morgan Wallen,https://open.spotify.com/track/1MX19fnaW1OUQog...,United States,top200,MOVE_DOWN,369274.000,"10 Dec 2021, 23:50","14 Jan 2018, 02:26",Morgan Wallen is a country music singer and so...,Morgan Wallen is a country music singer and so...,country,racist,male vocalist,the voice,racism,NaN,NaN,NaN,NaN,NaN,spotify:track:1MX19fnaW1OUQoglWTYaOo,0.635,0.772,7.000,-4.825,1.000,0.028,0.216,0.000,0.285,0.948,119.017,audio_features,1MX19fnaW1OUQoglWTYaOo,https://api.spotify.com/v1/tracks/1MX19fnaW1OU...,https://api.spotify.com/v1/audio-analysis/1MX1...,147893.000,4.000,27,47,country,country


In [23]:
## funciona pero tiene un poco de error
def generos(col):
    for key, value in bb.dict_gender.items():
        for k in key:
            for item in value:
                try:
                    if item in col.lower():
                        return key
                except:
                    return np.nan
    return np.nan

In [24]:
def generos_2(col):
    for key, value in bb.dict_gender.items():
        try:
            if value in col.lower():
                return key
        except:
            return np.nan
    return np.nan

In [25]:
df.head(1)

,title,rank,date,artist,url,region,chart,trend,streams,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,highest_rank,weeks_in_charts,music_genre,clean_music_genre
0,Bad and Boujee (feat. Lil Uzi Vert),1,2017-01-01,Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,United States,top200,SAME_POSITION,1371493.000,"30 Jan 2018, 04:41","26 Jun 2013, 14:20","Migos is a rap trio-group from Lawrenceville, ...","Migos is a rap trio-group from Lawrenceville, ...",Hip-Hop,trap,rap,hip hop,atlanta,trap,Hip-Hop,rap,2010s,meme rap,spotify:track:0M9ydKzuF3oZTfYYPfaGX1,0.926,0.666,11.000,-5.314,1.000,0.244,0.061,0.000,0.123,0.168,127.079,audio_features,0M9ydKzuF3oZTfYYPfaGX1,https://api.spotify.com/v1/tracks/0M9ydKzuF3oZ...,https://api.spotify.com/v1/audio-analysis/0M9y...,343150.000,4.000,1,429,hip-hop,hip-hop


In [26]:
#df["gender"] = df.apply(lambda x: generos(x['content'], bb.dict_gender), axis=1)
df["gender"] = df['summary'].apply(generos)

In [27]:
df['gender'].value_counts() / df.shape[0]

male         0.497
female       0.141
group        0.073
non-binary   0.030
Name: gender, dtype: float64

In [28]:
df.sample(5)

,title,rank,date,artist,url,region,chart,trend,streams,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,highest_rank,weeks_in_charts,music_genre,clean_music_genre,gender
451651,Moon Child,9,2017-03-23,The F16s,https://open.spotify.com/track/4TcrTb8W4lKkoi6...,United States,viral50,MOVE_DOWN,NaN,NaN,"16 Feb 2014, 09:51","An Electro Indie outfit formed in 2012, hailin...","An Electro Indie outfit formed in 2012, hailin...",indie rock,alternative,Indian,electronic,indie pop,NaN,NaN,NaN,NaN,NaN,spotify:track:5Y4vRBs4QCUloQ5W9Wb58j,0.606,0.713,9.000,-5.987,1.000,0.031,0.057,0.003,0.081,0.549,120.016,audio_features,5Y4vRBs4QCUloQ5W9Wb58j,https://api.spotify.com/v1/tracks/5Y4vRBs4QCUl...,https://api.spotify.com/v1/audio-analysis/5Y4v...,279841.000,4.000,8,8,indie rock,indie rock,NaN
163531,Goodbyes (feat. Young Thug),164,2020-07-20,Post Malone,https://open.spotify.com/track/0t3ZvGKlmYmVsDz...,United States,top200,MOVE_DOWN,227958.000,"01 Aug 2019, 13:10","07 Jun 2015, 09:50","Austin Richard Post (born July 4, 1995), known...","Austin Richard Post (born July 4, 1995), known...",Hip-Hop,rap,trap,cloud rap,pop,Hip-Hop,pop,2010s,2019,hip hop,spotify:track:0t3ZvGKlmYmVsDzBJAXK8C,0.548,0.653,5.000,-3.875,1.000,0.081,0.456,0.000,0.103,0.163,150.068,audio_features,0t3ZvGKlmYmVsDzBJAXK8C,https://api.spotify.com/v1/tracks/0t3ZvGKlmYmV...,https://api.spotify.com/v1/audio-analysis/0t3Z...,174853.000,4.000,95,105,hip-hop,hip-hop,male
389115,Lonely (with benny blanco),77,2021-02-02,Justin Bieber,https://open.spotify.com/track/4y4spB9m0Q6026K...,United States,top200,MOVE_DOWN,300938.000,"05 Aug 2022, 08:08","28 Oct 2008, 04:42","Justin Drew Bieber (born March 1, 1994 in Lond...","Justin Drew Bieber (born March 1, 1994 in Lond...",pop,black metal,justin bieber,rnb,Hip-Hop,sad,justin bieber,2020s,2020,pop,spotify:track:4y4spB9m0Q6026KfkAvy9Q,0.631,0.239,11.000,-7.071,0.000,0.040,0.864,0.000,0.116,0.093,79.859,audio_features,4y4spB9m0Q6026KfkAvy9Q,https://api.spotify.com/v1/tracks/4y4spB9m0Q60...,https://api.spotify.com/v1/audio-analysis/4y4s...,149297.000,4.000,6,155,pop,pop,group
159450,Moonlight,81,2019-05-06,XXXTENTACION,https://open.spotify.com/track/0JP9xo3adEtGSdU...,United States,top200,MOVE_DOWN,328079.000,"30 Jan 2019, 16:22","18 Dec 2016, 00:43","Jahseh Dwayne Onfroy (January 23, 1998 – June ...","Jahseh Dwayne Onfroy (January 23, 1998 – June ...",Hip-Hop,rap,cloud rap,trap,emo rap,rap,Hip-Hop,emo,hip hop,emo rap,spotify:track:0JP9xo3adEtGSdUEISiszL,0.921,0.537,9.000,-5.723,0.000,0.080,0.556,0.004,0.102,0.711,128.009,audio_features,0JP9xo3adEtGSdUEISiszL,https://api.spotify.com/v1/tracks/0JP9xo3adEtG...,https://api.spotify.com/v1/audio-analysis/0JP9...,135090.000,4.000,2,941,hip-hop,hip-hop,male
142498,Murder On My Mind,122,2019-01-11,YNW Melly,https://open.spotify.com/track/7eBqSVxrzQZtK2m...,United States,top200,MOVE_DOWN,313009.000,"19 Apr 2021, 12:56","09 Aug 2018, 20:38","Jamell Maurice Demons (born May 1, 1999), bett...","Jamell Maurice Demons (born May 1, 1999), bett...",rap,trap,Hip-Hop,rnb,hip hop,hip hop,trap,YNW Melly,rap,Horrorcore,spotify:track:7eBqSVxrzQZtK2mmgRG6lC,0.759,0.730,0.000,-7.985,0.000,0.052,0.145,0.000,0.110,0.740,115.007,audio_features,7eBqSVxrzQZtK2mmgRG6lC,https://api.spotify.com/v1/tracks/7eBqSVxrzQZt...,https://api.spotify.com/v1/audio-analysis/7eBq...,268434.000,4.000,3,866,rap,rap,male


In [29]:
df[df['artist'] == 'Kanye West'].to_csv('test.csv', sep = ';')

Age from artist

-- taking the data from the summary

In [30]:
df['birthday'] =  df['content'].apply(lambda x: re.findall(r'\w* \d*, \d{4}',str(x))).str[0]
df['birthday'].value_counts()

In [ ]:
df.sample()

# EDA

# Questions / hypothesis

Most streamed artists

In [ ]:
top10 = df.groupby(['artist'])['streams'].sum().reset_index().sort_values(by = 'streams', ascending = False).head(10)
top10['ratio'] = top10['streams'] * 100 / (df[~df['streams'].isnull()]['streams'].sum())
top10

Most streamed music genres

In [ ]:
top10genre = df.groupby(['aritist_genre_0'])['streams'].sum().reset_index().sort_values(by = 'streams', ascending = False).head(10)
top10genre['ratio'] = top10genre['streams'] * 100 / (df[~df['streams'].isnull()]['streams'].sum())
top10genre[top10genre['ratio'] > 0]

Artist evolution

In [ ]:
lista_artistas = list(top10['artist'])
df_top10 = df[df['artist'].isin(lista_artistas)]
df_top10.head()

In [ ]:
num_rows = 20
years = list(range(1990, 1990 + num_rows))
data_preproc = pd.DataFrame({
    'Year': years, 
    'A': np.random.randn(num_rows).cumsum(),
    'B': np.random.randn(num_rows).cumsum(),
    'C': np.random.randn(num_rows).cumsum(),
    'D': np.random.randn(num_rows).cumsum()})

In [ ]:
sns.lineplot(x='Year', y='value', hue='variable', 
             data=pd.melt(data_preproc, ['Year']))

In [ ]:
test = pd.pivot_table(df_top10[df_top10['chart'] == 'top200'][['date', 'artist', 'rank']], index = 'date', values = ['rank'], columns = 'artist').reset_index().droplevel(level=0, axis =1)
test.rename({'':'date'}, axis = 1, inplace=True)
print(test.shape)
test.head()

In [ ]:
sns.lineplot(x='Year', y='value', hue='variable', 
             data=pd.melt(data_preproc, ['Year']))

In [ ]:
sns.set(rc={'figure.figsize':(50,8.27)})

c = sns.lineplot(data = test, x = test['date'].unique().tolist(), y = 'Ariana Grande')
c.set_xticklabels(labels = test['date'].unique().tolist(), rotation=90)

Music genre evolution

In [ ]:
df

Genre distibution

How was the music during covid